# Proof of concept of working of AutoEncoder

In [1]:
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


class AutoEncoder(torch.nn.Module):

    def __init__(self, D_in, H, shared_decoder):
        # Encoder
        super(AutoEncoder, self).__init__()
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        # Decoder
        self.decoder = shared_decoder
        
    def forward(self, x):
        # Use encoder for encoding
        x = self.w1(x)
        x = self.activation(x)
        x = self.w2(x)
        
        # Decoder
        x = self.decoder(x)
        
        return x

In [2]:
# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de', 'nl'], 'trgt': ['en']}


# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Create toy train dataset
rows, cols = 10, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = ae(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')
# Train
for i in range(1000):
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_ae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
    
# Report outcome after training
reproduction = ae(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Created: Nonlinear GAN.
Input data:
 tensor([[0.2983, 0.3470, 0.9130, 0.6330, 0.3939, 0.0118, 0.6965, 0.3947],
        [0.2274, 0.5950, 0.7225, 0.5266, 0.4859, 0.4181, 0.9203, 0.7723],
        [0.7886, 0.3672, 0.2622, 0.3294, 0.5694, 0.3461, 0.3176, 0.8671],
        [0.7754, 0.1192, 0.4909, 0.1797, 0.0425, 0.4058, 0.8058, 0.8497],
        [0.7561, 0.4451, 0.2330, 0.6630, 0.7230, 0.6286, 0.8799, 0.8165],
        [0.1627, 0.7855, 0.6362, 0.2538, 0.0212, 0.5719, 0.6072, 0.7573],
        [0.8950, 0.5215, 0.1934, 0.2554, 0.5900, 0.5347, 0.7281, 0.2351],
        [0.1041, 0.0178, 0.0954, 0.1840, 0.8831, 0.3684, 0.3370, 0.7462],
        [0.5881, 0.7652, 0.0871, 0.6783, 0.3061, 0.4146, 0.0310, 0.3446],
        [0.8423, 0.2590, 0.8784, 0.8271, 0.7390, 0.9448, 0.1501, 0.1027]])
Reproduced data:
 tensor([[ 0.0471,  0.0008, -0.0290, -0.0195,  0.0451, -0.0366, -0.0066,  0.0844],
        [ 0.0439,  0.0182, -0.0251, -0.0094,  0.0439, -0.0450, -0.0129,  0.0896],
        [ 0.0156,  0.0315, -0.0309, -0.0

Epoch 282 Loss: 4.0070493e-08
Epoch 283 Loss: 1.5866334e-08
Epoch 284 Loss: 1.2227289e-08
Epoch 285 Loss: 3.0034816e-08
Epoch 286 Loss: 1.3337981e-09
Epoch 287 Loss: 2.2054204e-08
Epoch 288 Loss: 1.169885e-08
Epoch 289 Loss: 5.3365214e-09
Epoch 290 Loss: 1.9126654e-08
Epoch 291 Loss: 1.7546851e-09
Epoch 292 Loss: 1.1274327e-08
Epoch 293 Loss: 9.304006e-09
Epoch 294 Loss: 1.4224499e-09
Epoch 295 Loss: 1.13607665e-08
Epoch 296 Loss: 2.2464384e-09
Epoch 297 Loss: 4.722641e-09
Epoch 298 Loss: 7.3869857e-09
Epoch 299 Loss: 4.700494e-10
Epoch 300 Loss: 6.243146e-09
Epoch 301 Loss: 3.0507668e-09
Epoch 302 Loss: 1.3782763e-09
Epoch 303 Loss: 5.105689e-09
Epoch 304 Loss: 6.5036565e-10
Epoch 305 Loss: 2.5393418e-09
Epoch 306 Loss: 3.0451293e-09
Epoch 307 Loss: 2.35045e-10
Epoch 308 Loss: 2.8480538e-09
Epoch 309 Loss: 1.3280678e-09
Epoch 310 Loss: 6.367298e-10
Epoch 311 Loss: 2.3217168e-09
Epoch 312 Loss: 3.921507e-10
Epoch 313 Loss: 1.0375016e-09
Epoch 314 Loss: 1.493641e-09
Epoch 315 Loss: 1.13

Epoch 608 Loss: 1.35394845e-14
Epoch 609 Loss: 8.80138e-15
Epoch 610 Loss: 1.1619264e-14
Epoch 611 Loss: 9.92406e-15
Epoch 612 Loss: 1.5193369e-14
Epoch 613 Loss: 7.671728e-15
Epoch 614 Loss: 1.2529951e-14
Epoch 615 Loss: 1.1210445e-14
Epoch 616 Loss: 9.153496e-15
Epoch 617 Loss: 1.5122366e-14
Epoch 618 Loss: 8.178322e-15
Epoch 619 Loss: 2.9357908e-14
Epoch 620 Loss: 1.0293025e-14
Epoch 621 Loss: 1.10139e-14
Epoch 622 Loss: 8.655316e-15
Epoch 623 Loss: 9.8620436e-15
Epoch 624 Loss: 1.3994545e-14
Epoch 625 Loss: 1.0334583e-14
Epoch 626 Loss: 7.371371e-15
Epoch 627 Loss: 9.363744e-15
Epoch 628 Loss: 5.7818444e-15
Epoch 629 Loss: 8.42243e-15
Epoch 630 Loss: 5.459912e-15
Epoch 631 Loss: 6.4178263e-15
Epoch 632 Loss: 5.818653e-15
Epoch 633 Loss: 4.3075352e-15
Epoch 634 Loss: 4.840442e-15
Epoch 635 Loss: 5.1412e-15
Epoch 636 Loss: 4.295837e-15
Epoch 637 Loss: 6.640489e-15
Epoch 638 Loss: 3.578442e-15
Epoch 639 Loss: 4.9840884e-15
Epoch 640 Loss: 4.1133436e-15
Epoch 641 Loss: 6.065157e-15
Epo

Epoch 889 Loss: 5.8226655e-05
Epoch 890 Loss: 5.457269e-05
Epoch 891 Loss: 5.1015733e-05
Epoch 892 Loss: 4.812494e-05
Epoch 893 Loss: 4.488728e-05
Epoch 894 Loss: 4.1984713e-05
Epoch 895 Loss: 4.017224e-05
Epoch 896 Loss: 3.7977574e-05
Epoch 897 Loss: 3.5938476e-05
Epoch 898 Loss: 3.417442e-05
Epoch 899 Loss: 3.2697528e-05
Epoch 900 Loss: 3.1006603e-05
Epoch 901 Loss: 2.993331e-05
Epoch 902 Loss: 2.8566055e-05
Epoch 903 Loss: 2.7232603e-05
Epoch 904 Loss: 2.606787e-05
Epoch 905 Loss: 2.480254e-05
Epoch 906 Loss: 2.3532479e-05
Epoch 907 Loss: 2.2571574e-05
Epoch 908 Loss: 2.1415446e-05
Epoch 909 Loss: 2.025663e-05
Epoch 910 Loss: 1.9190858e-05
Epoch 911 Loss: 1.8154853e-05
Epoch 912 Loss: 1.7153321e-05
Epoch 913 Loss: 1.6329657e-05
Epoch 914 Loss: 1.5462885e-05
Epoch 915 Loss: 1.4724896e-05
Epoch 916 Loss: 1.40050815e-05
Epoch 917 Loss: 1.3260976e-05
Epoch 918 Loss: 1.2547853e-05
Epoch 919 Loss: 1.1903854e-05
Epoch 920 Loss: 1.1271226e-05
Epoch 921 Loss: 1.0655562e-05
Epoch 922 Loss: 1.

# Proof of concept of working of NoisyAutoEncoder

In [1]:
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


def bound_noise(shape, min_=-0.05, max_=0.05):
    return (min_ - max_) * torch.rand(shape) + max_



class NoisyAutoEncoder(torch.nn.Module):

    def __init__(self, shared_encoder, D_in, H, D_out):
        super(NoisyAutoEncoder, self).__init__()
        # Encoder
        self.encoder = shared_encoder
        
        # Decoder
        self.w1 = torch.nn.Linear(D_in, H)
        self.w2 = torch.nn.Linear(H, H)
        
        self.activation = torch.nn.functional.relu
        
        self.noise = bound_noise
        
    def forward(self, x):
        # Use encoder for encoding, but reproduce original encoder input in the presence of noise
        # Encoder
        x = self.encoder(x)
        
        # Noisy Decoder
        x = x + self.noise(x.shape)
        x = self.w1(x)
        x = x + self.noise(x.shape)
        x = self.activation(x)
        x = self.w2(x)
        
        return x

In [2]:
# Network param settings
embedding_dim = 8
internal_dim = 8
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de', 'nl'], 'trgt': ['en']}


# Architectures
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

nae = NoisyAutoEncoder(net.generator.encoders['de'], embedding_dim, internal_dim, embedding_dim)
optimizer_nae = torch.optim.Adam(nae.parameters(),
                                         lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                         weight_decay=0., amsgrad=False)

# Create toy train dataset
rows, cols = 10, embedding_dim
train_data = torch.rand(rows, cols)

# Report outcome before training
reproduction = nae(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')
# Train
for i in range(1000):
    nae.train()
    optimizer_nae.zero_grad()
    y_pred = nae(train_data)
    loss = AutoEncoderLoss(y_pred, train_data)
    loss.backward()
    optimizer_nae.step()
    print('Epoch', i, 'Loss:', loss.detach().numpy())
    
# Report outcome after training
reproduction = nae(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Created: Nonlinear GAN.
Input data:
 tensor([[0.6955, 0.8632, 0.9169, 0.6074, 0.0979, 0.8319, 0.9729, 0.2390],
        [0.9202, 0.2667, 0.7894, 0.4576, 0.5624, 0.0496, 0.8259, 0.6745],
        [0.6373, 0.3681, 0.5543, 0.2419, 0.6370, 0.4306, 0.1784, 0.4197],
        [0.5217, 0.5681, 0.8534, 0.1555, 0.6248, 0.5567, 0.3933, 0.0390],
        [0.6081, 0.9980, 0.8363, 0.6430, 0.1053, 0.2996, 0.3614, 0.2882],
        [0.5352, 0.0141, 0.5443, 0.5844, 0.0973, 0.5414, 0.0734, 0.8997],
        [0.2985, 0.1600, 0.0263, 0.4869, 0.6624, 0.5031, 0.1481, 0.6310],
        [0.8992, 0.6558, 0.9977, 0.8203, 0.5338, 0.1110, 0.5505, 0.4464],
        [0.3392, 0.6827, 0.5428, 0.9617, 0.3691, 0.9315, 0.9865, 0.4138],
        [0.4521, 0.4637, 0.1672, 0.4879, 0.9044, 0.3850, 0.5389, 0.6574]])
Reproduced data:
 tensor([[-0.1297, -0.2618, -0.1686, -0.3685, -0.0350, -0.0147,  0.0103,  0.1255],
        [-0.1476, -0.2815, -0.1329, -0.4041, -0.0597, -0.0091, -0.0190,  0.1474],
        [-0.1373, -0.2845, -0.1582, -0.3

Epoch 259 Loss: 0.021854956
Epoch 260 Loss: 0.020957064
Epoch 261 Loss: 0.021476872
Epoch 262 Loss: 0.02046872
Epoch 263 Loss: 0.020336252
Epoch 264 Loss: 0.020723859
Epoch 265 Loss: 0.021032726
Epoch 266 Loss: 0.021023165
Epoch 267 Loss: 0.020134443
Epoch 268 Loss: 0.020372901
Epoch 269 Loss: 0.020631434
Epoch 270 Loss: 0.019985732
Epoch 271 Loss: 0.021047328
Epoch 272 Loss: 0.020237638
Epoch 273 Loss: 0.020999392
Epoch 274 Loss: 0.020233897
Epoch 275 Loss: 0.020345729
Epoch 276 Loss: 0.020266509
Epoch 277 Loss: 0.02095438
Epoch 278 Loss: 0.020312855
Epoch 279 Loss: 0.020686839
Epoch 280 Loss: 0.019896638
Epoch 281 Loss: 0.020733308
Epoch 282 Loss: 0.019833826
Epoch 283 Loss: 0.020243336
Epoch 284 Loss: 0.02052216
Epoch 285 Loss: 0.019617056
Epoch 286 Loss: 0.018912235
Epoch 287 Loss: 0.019555857
Epoch 288 Loss: 0.019429436
Epoch 289 Loss: 0.019471504
Epoch 290 Loss: 0.019262156
Epoch 291 Loss: 0.019316949
Epoch 292 Loss: 0.020931805
Epoch 293 Loss: 0.0200709
Epoch 294 Loss: 0.0201926

Epoch 637 Loss: 0.018206215
Epoch 638 Loss: 0.018078601
Epoch 639 Loss: 0.017991403
Epoch 640 Loss: 0.01797359
Epoch 641 Loss: 0.017908126
Epoch 642 Loss: 0.018106861
Epoch 643 Loss: 0.017735794
Epoch 644 Loss: 0.017807392
Epoch 645 Loss: 0.017402243
Epoch 646 Loss: 0.018154034
Epoch 647 Loss: 0.017727003
Epoch 648 Loss: 0.017649341
Epoch 649 Loss: 0.01771266
Epoch 650 Loss: 0.018244602
Epoch 651 Loss: 0.017965706
Epoch 652 Loss: 0.017482443
Epoch 653 Loss: 0.01825071
Epoch 654 Loss: 0.017818663
Epoch 655 Loss: 0.017759722
Epoch 656 Loss: 0.017609444
Epoch 657 Loss: 0.01827849
Epoch 658 Loss: 0.01778496
Epoch 659 Loss: 0.017476553
Epoch 660 Loss: 0.017425502
Epoch 661 Loss: 0.017489418
Epoch 662 Loss: 0.017243238
Epoch 663 Loss: 0.018133748
Epoch 664 Loss: 0.017614786
Epoch 665 Loss: 0.017579423
Epoch 666 Loss: 0.01766995
Epoch 667 Loss: 0.017967489
Epoch 668 Loss: 0.01728049
Epoch 669 Loss: 0.017232312
Epoch 670 Loss: 0.017896712
Epoch 671 Loss: 0.017923275
Epoch 672 Loss: 0.018008012

# Proof of concept of working of linear-GAN

In [7]:
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1


# Architecture
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages, mode='linear')

# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}

optimizers['gen']['de'] = torch.optim.Adam(net.generator.parameters(),
                                           lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                           weight_decay=0.001, amsgrad=False)
    
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

# Loss function
criterion = torch.nn.BCELoss()


net.generator.beta = 0.00001 # Stepsize in making linear generator orthogonal
for i in range(10000):
    # Perform incremental orthogonalization of linear generator
    net.generator.orthogonalize()


# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)


# Report outcome before training
reproduction = net.generator(train_data)
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(5000):

    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data)  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data)
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()
    net.generator.orthogonalize()
    
    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy())
    
    
# Report outcome after training
reproduction = net.generator(train_data)
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Created: Nonlinear GAN.
Input data:
 tensor([[0.1826, 0.8896, 0.3567, 0.6658, 0.6822, 0.8601, 0.5014, 0.1657],
        [0.0116, 0.9206, 0.7645, 0.8923, 0.8268, 0.5848, 0.6091, 0.5396],
        [0.3141, 0.2614, 0.5044, 0.8782, 0.6877, 0.2298, 0.5058, 0.1705],
        [0.3924, 0.1006, 0.6340, 0.1144, 0.1850, 0.0651, 0.4640, 0.2693],
        [0.9957, 0.1258, 0.5432, 0.9559, 0.8562, 0.3954, 0.6868, 0.6816],
        [0.5063, 0.2726, 0.3335, 0.2488, 0.9884, 0.7462, 0.8408, 0.4053],
        [0.3269, 0.2256, 0.1704, 0.6408, 0.6585, 0.9112, 0.3538, 0.8383],
        [0.5630, 0.1890, 0.7699, 0.9955, 0.2616, 0.0101, 0.7342, 0.8128],
        [0.8933, 0.1517, 0.5592, 0.9848, 0.2888, 0.5655, 0.5483, 0.9103],
        [0.5525, 0.9508, 0.2539, 0.7247, 0.4768, 0.2086, 0.8041, 0.0549]])
Reproduced data:
 tensor([[-0.3026, -0.2987, -0.1345, -0.3790, -0.0061,  0.5323, -0.0619,  0.3717],
        [-0.3597, -0.3648, -0.3890, -0.2417,  0.0325,  0.7858, -0.2298,  0.5255],
        [-0.1382, -0.3419, -0.3013, -0.1

Epoch 126 Generator loss: 1.8410009 Discriminator loss: 0.45852718
Epoch 127 Generator loss: 1.821983 Discriminator loss: 0.4686597
Epoch 128 Generator loss: 1.8048643 Discriminator loss: 0.47750488
Epoch 129 Generator loss: 1.7899662 Discriminator loss: 0.48504776
Epoch 130 Generator loss: 1.7778065 Discriminator loss: 0.49115816
Epoch 131 Generator loss: 1.7689259 Discriminator loss: 0.4958927
Epoch 132 Generator loss: 1.7630217 Discriminator loss: 0.49940458
Epoch 133 Generator loss: 1.7597955 Discriminator loss: 0.50158465
Epoch 134 Generator loss: 1.7586906 Discriminator loss: 0.50252
Epoch 135 Generator loss: 1.7589861 Discriminator loss: 0.50231427
Epoch 136 Generator loss: 1.7594811 Discriminator loss: 0.5010294
Epoch 137 Generator loss: 1.76033 Discriminator loss: 0.49887735
Epoch 138 Generator loss: 1.7619711 Discriminator loss: 0.49595943
Epoch 139 Generator loss: 1.7645645 Discriminator loss: 0.4924272
Epoch 140 Generator loss: 1.7679603 Discriminator loss: 0.488273
Epoch 1

Epoch 264 Generator loss: 1.755196 Discriminator loss: 0.24717636
Epoch 265 Generator loss: 1.7609737 Discriminator loss: 0.24637607
Epoch 266 Generator loss: 1.7694362 Discriminator loss: 0.24566309
Epoch 267 Generator loss: 1.7795368 Discriminator loss: 0.2449718
Epoch 268 Generator loss: 1.7893307 Discriminator loss: 0.24422288
Epoch 269 Generator loss: 1.7981498 Discriminator loss: 0.24350068
Epoch 270 Generator loss: 1.8079134 Discriminator loss: 0.24258241
Epoch 271 Generator loss: 1.8188232 Discriminator loss: 0.24152169
Epoch 272 Generator loss: 1.8319368 Discriminator loss: 0.24032398
Epoch 273 Generator loss: 1.8453239 Discriminator loss: 0.23891838
Epoch 274 Generator loss: 1.858866 Discriminator loss: 0.23745768
Epoch 275 Generator loss: 1.8725102 Discriminator loss: 0.23610151
Epoch 276 Generator loss: 1.8857685 Discriminator loss: 0.2348161
Epoch 277 Generator loss: 1.8983696 Discriminator loss: 0.23359375
Epoch 278 Generator loss: 1.9123615 Discriminator loss: 0.2324225


Epoch 418 Generator loss: 2.2035654 Discriminator loss: 0.23001738
Epoch 419 Generator loss: 2.1657116 Discriminator loss: 0.2455883
Epoch 420 Generator loss: 2.128092 Discriminator loss: 0.26252457
Epoch 421 Generator loss: 2.0920558 Discriminator loss: 0.28095534
Epoch 422 Generator loss: 2.0573077 Discriminator loss: 0.30063695
Epoch 423 Generator loss: 2.0234458 Discriminator loss: 0.32148787
Epoch 424 Generator loss: 1.9909217 Discriminator loss: 0.34338242
Epoch 425 Generator loss: 1.9626505 Discriminator loss: 0.3661856
Epoch 426 Generator loss: 1.936697 Discriminator loss: 0.38964176
Epoch 427 Generator loss: 1.9122165 Discriminator loss: 0.41347584
Epoch 428 Generator loss: 1.8877052 Discriminator loss: 0.43753234
Epoch 429 Generator loss: 1.8645382 Discriminator loss: 0.4619012
Epoch 430 Generator loss: 1.8432243 Discriminator loss: 0.48617283
Epoch 431 Generator loss: 1.8255188 Discriminator loss: 0.50998473
Epoch 432 Generator loss: 1.8113413 Discriminator loss: 0.53299147


Epoch 570 Generator loss: 2.5964522 Discriminator loss: 0.13687378
Epoch 571 Generator loss: 2.5947416 Discriminator loss: 0.13612959
Epoch 572 Generator loss: 2.5919845 Discriminator loss: 0.13580719
Epoch 573 Generator loss: 2.5891805 Discriminator loss: 0.13578682
Epoch 574 Generator loss: 2.5866795 Discriminator loss: 0.13610646
Epoch 575 Generator loss: 2.5830717 Discriminator loss: 0.1368407
Epoch 576 Generator loss: 2.5773153 Discriminator loss: 0.13799502
Epoch 577 Generator loss: 2.5684705 Discriminator loss: 0.13952862
Epoch 578 Generator loss: 2.555338 Discriminator loss: 0.1414307
Epoch 579 Generator loss: 2.5379007 Discriminator loss: 0.14377537
Epoch 580 Generator loss: 2.5173097 Discriminator loss: 0.1466408
Epoch 581 Generator loss: 2.4937227 Discriminator loss: 0.15002695
Epoch 582 Generator loss: 2.4682376 Discriminator loss: 0.15404595
Epoch 583 Generator loss: 2.442294 Discriminator loss: 0.15868656
Epoch 584 Generator loss: 2.4150357 Discriminator loss: 0.1640001
E

...

Epoch 4329 Generator loss: 2.6287189 Discriminator loss: 0.094021946
Epoch 4330 Generator loss: 2.6374073 Discriminator loss: 0.0928447
Epoch 4331 Generator loss: 2.646465 Discriminator loss: 0.09163249
Epoch 4332 Generator loss: 2.6566658 Discriminator loss: 0.09042388
Epoch 4333 Generator loss: 2.6682148 Discriminator loss: 0.08910079
Epoch 4334 Generator loss: 2.67838 Discriminator loss: 0.08774795
Epoch 4335 Generator loss: 2.68764 Discriminator loss: 0.08633531
Epoch 4336 Generator loss: 2.6954 Discriminator loss: 0.08485258
Epoch 4337 Generator loss: 2.7081428 Discriminator loss: 0.08343728
Epoch 4338 Generator loss: 2.7254086 Discriminator loss: 0.082138255
Epoch 4339 Generator loss: 2.743927 Discriminator loss: 0.0810261
Epoch 4340 Generator loss: 2.761119 Discriminator loss: 0.07997112
Epoch 4341 Generator loss: 2.7720275 Discriminator loss: 0.07906597
Epoch 4342 Generator loss: 2.7762785 Discriminator loss: 0.07828231
Epoch 4343 Generator loss: 2.77984 Discriminator loss: 0.0

Epoch 4459 Generator loss: 2.203094 Discriminator loss: 0.1894898
Epoch 4460 Generator loss: 2.2103288 Discriminator loss: 0.18821464
Epoch 4461 Generator loss: 2.214928 Discriminator loss: 0.18720892
Epoch 4462 Generator loss: 2.2180142 Discriminator loss: 0.18591201
Epoch 4463 Generator loss: 2.2215962 Discriminator loss: 0.18445255
Epoch 4464 Generator loss: 2.2305539 Discriminator loss: 0.18246454
Epoch 4465 Generator loss: 2.2452118 Discriminator loss: 0.17986743
Epoch 4466 Generator loss: 2.2585287 Discriminator loss: 0.1775032
Epoch 4467 Generator loss: 2.2700622 Discriminator loss: 0.17545435
Epoch 4468 Generator loss: 2.2805982 Discriminator loss: 0.1735398
Epoch 4469 Generator loss: 2.294137 Discriminator loss: 0.17198966
Epoch 4470 Generator loss: 2.3129077 Discriminator loss: 0.17079519
Epoch 4471 Generator loss: 2.3307912 Discriminator loss: 0.16982684
Epoch 4472 Generator loss: 2.3482785 Discriminator loss: 0.16915937
Epoch 4473 Generator loss: 2.3647685 Discriminator los

Epoch 4585 Generator loss: 1.9785124 Discriminator loss: 0.24142163
Epoch 4586 Generator loss: 1.955953 Discriminator loss: 0.24586883
Epoch 4587 Generator loss: 1.9422127 Discriminator loss: 0.24885932
Epoch 4588 Generator loss: 1.9302076 Discriminator loss: 0.2502617
Epoch 4589 Generator loss: 1.9126253 Discriminator loss: 0.25029305
Epoch 4590 Generator loss: 1.8978732 Discriminator loss: 0.24995752
Epoch 4591 Generator loss: 1.8932524 Discriminator loss: 0.24913725
Epoch 4592 Generator loss: 1.8982546 Discriminator loss: 0.24715939
Epoch 4593 Generator loss: 1.9070263 Discriminator loss: 0.24388626
Epoch 4594 Generator loss: 1.9181726 Discriminator loss: 0.23920813
Epoch 4595 Generator loss: 1.9324381 Discriminator loss: 0.2329117
Epoch 4596 Generator loss: 1.9542506 Discriminator loss: 0.22573066
Epoch 4597 Generator loss: 1.9839483 Discriminator loss: 0.21809158
Epoch 4598 Generator loss: 2.0203304 Discriminator loss: 0.20927179
Epoch 4599 Generator loss: 2.0638366 Discriminator 

Epoch 4711 Generator loss: 2.1406274 Discriminator loss: 0.16959254
Epoch 4712 Generator loss: 2.133358 Discriminator loss: 0.16945817
Epoch 4713 Generator loss: 2.1262684 Discriminator loss: 0.16897365
Epoch 4714 Generator loss: 2.124812 Discriminator loss: 0.16827175
Epoch 4715 Generator loss: 2.1319857 Discriminator loss: 0.16722491
Epoch 4716 Generator loss: 2.1409183 Discriminator loss: 0.16585077
Epoch 4717 Generator loss: 2.1437576 Discriminator loss: 0.16426815
Epoch 4718 Generator loss: 2.1439695 Discriminator loss: 0.1626313
Epoch 4719 Generator loss: 2.1489255 Discriminator loss: 0.16096893
Epoch 4720 Generator loss: 2.1610441 Discriminator loss: 0.15905246
Epoch 4721 Generator loss: 2.178044 Discriminator loss: 0.15706801
Epoch 4722 Generator loss: 2.1980033 Discriminator loss: 0.15494882
Epoch 4723 Generator loss: 2.216044 Discriminator loss: 0.1528355
Epoch 4724 Generator loss: 2.2318184 Discriminator loss: 0.15072711
Epoch 4725 Generator loss: 2.2446802 Discriminator los

Epoch 4854 Generator loss: 2.4175131 Discriminator loss: 0.15497689
Epoch 4855 Generator loss: 2.4095778 Discriminator loss: 0.15534225
Epoch 4856 Generator loss: 2.4018018 Discriminator loss: 0.15609463
Epoch 4857 Generator loss: 2.3911743 Discriminator loss: 0.15710256
Epoch 4858 Generator loss: 2.377543 Discriminator loss: 0.15829849
Epoch 4859 Generator loss: 2.3630397 Discriminator loss: 0.15974537
Epoch 4860 Generator loss: 2.3511653 Discriminator loss: 0.16149537
Epoch 4861 Generator loss: 2.3410113 Discriminator loss: 0.16354182
Epoch 4862 Generator loss: 2.3290257 Discriminator loss: 0.16590276
Epoch 4863 Generator loss: 2.3112545 Discriminator loss: 0.1685301
Epoch 4864 Generator loss: 2.2882504 Discriminator loss: 0.17139634
Epoch 4865 Generator loss: 2.2643495 Discriminator loss: 0.17460507
Epoch 4866 Generator loss: 2.2412539 Discriminator loss: 0.17778638
Epoch 4867 Generator loss: 2.219378 Discriminator loss: 0.18111429
Epoch 4868 Generator loss: 2.199082 Discriminator l

# Proof of concept of working of non-linear-GAN

In [1]:
import torch
from model import gan


def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 300
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1


# Architecture
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])

# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in languages['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                               lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                               weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

# Loss function
criterion = torch.nn.BCELoss()


# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)


# Report outcome before training
reproduction = net.generator(train_data, 'de')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(10000):

    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()

    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy())
    
    
# Report outcome after training
reproduction = net.generator(train_data, 'de')
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Created: Nonlinear GAN.
Input data:
 tensor([[0.1077, 0.3201, 0.6540, 0.7947, 0.8013, 0.3829, 0.2114, 0.8464],
        [0.8019, 0.1613, 0.1681, 0.0701, 0.6075, 0.2847, 0.2947, 0.2785],
        [0.5296, 0.4515, 0.7441, 0.0518, 0.7390, 0.5385, 0.7547, 0.2864],
        [0.7995, 0.3041, 0.9838, 0.2383, 0.0075, 0.4505, 0.3445, 0.8438],
        [0.0491, 0.2351, 0.0521, 0.6553, 0.6114, 0.1229, 0.3831, 0.3020],
        [0.6174, 0.2246, 0.0169, 0.9962, 0.4257, 0.2230, 0.4192, 0.6686],
        [0.4811, 0.8337, 0.8484, 0.4929, 0.6979, 0.7583, 0.0215, 0.4607],
        [0.5414, 0.6983, 0.8480, 0.7351, 0.0169, 0.7176, 0.3790, 0.9204],
        [0.2412, 0.3828, 0.1318, 0.4446, 0.1073, 0.6599, 0.7588, 0.5349],
        [0.5819, 0.4118, 0.1946, 0.7230, 0.3500, 0.6720, 0.7518, 0.7958]])
Reproduced data:
 tensor([[ 0.0084, -0.0331, -0.0882,  0.0537,  0.0264, -0.1147, -0.0118, -0.0585],
        [ 0.0312, -0.0462, -0.1016,  0.0288, -0.0077, -0.1068,  0.0138, -0.0606],
        [ 0.0181, -0.0396, -0.1009,  0.0

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target

Epoch 26 Generator loss: 0.8523305 Discriminator loss: 0.92857707
Epoch 27 Generator loss: 0.81996965 Discriminator loss: 0.8965899
Epoch 28 Generator loss: 0.81197226 Discriminator loss: 0.85964316
Epoch 29 Generator loss: 0.8289402 Discriminator loss: 0.8180481
Epoch 30 Generator loss: 0.87669724 Discriminator loss: 0.77060854
Epoch 31 Generator loss: 0.95659196 Discriminator loss: 0.7179686
Epoch 32 Generator loss: 1.0744777 Discriminator loss: 0.6596318
Epoch 33 Generator loss: 1.2280376 Discriminator loss: 0.5984001
Epoch 34 Generator loss: 1.4023252 Discriminator loss: 0.5390767
Epoch 35 Generator loss: 1.523894 Discriminator loss: 0.49619207
Epoch 36 Generator loss: 1.5401478 Discriminator loss: 0.4746704
Epoch 37 Generator loss: 1.468873 Discriminator loss: 0.47050214
Epoch 38 Generator loss: 1.3263606 Discriminator loss: 0.48586613
Epoch 39 Generator loss: 1.2046053 Discriminator loss: 0.50508165
Epoch 40 Generator loss: 1.0972706 Discriminator loss: 0.5301818
Epoch 41 Generat

Epoch 176 Generator loss: 0.9038993 Discriminator loss: 0.6769689
Epoch 177 Generator loss: 0.93560296 Discriminator loss: 0.6752923
Epoch 178 Generator loss: 0.9384796 Discriminator loss: 0.6816209
Epoch 179 Generator loss: 0.91264665 Discriminator loss: 0.6958614
Epoch 180 Generator loss: 0.865794 Discriminator loss: 0.7163003
Epoch 181 Generator loss: 0.804718 Discriminator loss: 0.7424435
Epoch 182 Generator loss: 0.75007844 Discriminator loss: 0.76684564
Epoch 183 Generator loss: 0.7261495 Discriminator loss: 0.77663016
Epoch 184 Generator loss: 0.72674596 Discriminator loss: 0.7749494
Epoch 185 Generator loss: 0.7286848 Discriminator loss: 0.7721241
Epoch 186 Generator loss: 0.72118795 Discriminator loss: 0.77287686
Epoch 187 Generator loss: 0.70075685 Discriminator loss: 0.77948004
Epoch 188 Generator loss: 0.67137927 Discriminator loss: 0.7905008
Epoch 189 Generator loss: 0.64118063 Discriminator loss: 0.8023678
Epoch 190 Generator loss: 0.61472297 Discriminator loss: 0.8137161

Epoch 314 Generator loss: 0.690501 Discriminator loss: 0.73436266
Epoch 315 Generator loss: 0.6373027 Discriminator loss: 0.7486462
Epoch 316 Generator loss: 0.59275234 Discriminator loss: 0.7630833
Epoch 317 Generator loss: 0.5644467 Discriminator loss: 0.7729192
Epoch 318 Generator loss: 0.5562344 Discriminator loss: 0.77473366
Epoch 319 Generator loss: 0.56664604 Discriminator loss: 0.7685049
Epoch 320 Generator loss: 0.5932713 Discriminator loss: 0.7553823
Epoch 321 Generator loss: 0.63312596 Discriminator loss: 0.73736733
Epoch 322 Generator loss: 0.6826289 Discriminator loss: 0.7166541
Epoch 323 Generator loss: 0.7357145 Discriminator loss: 0.69620144
Epoch 324 Generator loss: 0.7843909 Discriminator loss: 0.6794168
Epoch 325 Generator loss: 0.81605875 Discriminator loss: 0.66993386
Epoch 326 Generator loss: 0.82455957 Discriminator loss: 0.6696566
Epoch 327 Generator loss: 0.8073226 Discriminator loss: 0.6790812
Epoch 328 Generator loss: 0.7720248 Discriminator loss: 0.6959444
E

...

Epoch 9378 Generator loss: 0.77009207 Discriminator loss: 0.6143857
Epoch 9379 Generator loss: 0.76723415 Discriminator loss: 0.61143243
Epoch 9380 Generator loss: 0.7739685 Discriminator loss: 0.602404
Epoch 9381 Generator loss: 0.77021414 Discriminator loss: 0.5984335
Epoch 9382 Generator loss: 0.78121555 Discriminator loss: 0.5872053
Epoch 9383 Generator loss: 0.7872587 Discriminator loss: 0.5801033
Epoch 9384 Generator loss: 0.7835878 Discriminator loss: 0.579374
Epoch 9385 Generator loss: 0.79059327 Discriminator loss: 0.57456344
Epoch 9386 Generator loss: 0.7963211 Discriminator loss: 0.57045126
Epoch 9387 Generator loss: 0.7976786 Discriminator loss: 0.57077134
Epoch 9388 Generator loss: 0.80470574 Discriminator loss: 0.56749934
Epoch 9389 Generator loss: 0.81364214 Discriminator loss: 0.5625279
Epoch 9390 Generator loss: 0.81843394 Discriminator loss: 0.56037605
Epoch 9391 Generator loss: 0.82784283 Discriminator loss: 0.5540297
Epoch 9392 Generator loss: 0.8380445 Discriminato

Epoch 9519 Generator loss: 0.98005235 Discriminator loss: 0.64212734
Epoch 9520 Generator loss: 0.94400567 Discriminator loss: 0.65782696
Epoch 9521 Generator loss: 0.95490617 Discriminator loss: 0.65911865
Epoch 9522 Generator loss: 0.9767569 Discriminator loss: 0.6601535
Epoch 9523 Generator loss: 0.9978841 Discriminator loss: 0.65987486
Epoch 9524 Generator loss: 0.9844707 Discriminator loss: 0.6663112
Epoch 9525 Generator loss: 0.9464613 Discriminator loss: 0.6799862
Epoch 9526 Generator loss: 0.88491327 Discriminator loss: 0.7018119
Epoch 9527 Generator loss: 0.843225 Discriminator loss: 0.7209066
Epoch 9528 Generator loss: 0.81804574 Discriminator loss: 0.7335762
Epoch 9529 Generator loss: 0.79704714 Discriminator loss: 0.7451558
Epoch 9530 Generator loss: 0.8117196 Discriminator loss: 0.7376777
Epoch 9531 Generator loss: 0.83722955 Discriminator loss: 0.7196836
Epoch 9532 Generator loss: 0.8538054 Discriminator loss: 0.70684105
Epoch 9533 Generator loss: 0.86286104 Discriminator

Epoch 9670 Generator loss: 0.7904994 Discriminator loss: 0.6261226
Epoch 9671 Generator loss: 0.802807 Discriminator loss: 0.61626446
Epoch 9672 Generator loss: 0.79320896 Discriminator loss: 0.61928964
Epoch 9673 Generator loss: 0.7624073 Discriminator loss: 0.6258117
Epoch 9674 Generator loss: 0.77130306 Discriminator loss: 0.616504
Epoch 9675 Generator loss: 0.8489524 Discriminator loss: 0.58625656
Epoch 9676 Generator loss: 0.80801326 Discriminator loss: 0.5917297
Epoch 9677 Generator loss: 0.8018185 Discriminator loss: 0.5861755
Epoch 9678 Generator loss: 0.8554741 Discriminator loss: 0.56024206
Epoch 9679 Generator loss: 0.803662 Discriminator loss: 0.5731068
Epoch 9680 Generator loss: 0.82688606 Discriminator loss: 0.55729246
Epoch 9681 Generator loss: 0.7383421 Discriminator loss: 0.58918244
Epoch 9682 Generator loss: 0.5804376 Discriminator loss: 0.6819821
Epoch 9683 Generator loss: 0.48519963 Discriminator loss: 0.7476598
Epoch 9684 Generator loss: 0.45734674 Discriminator lo

Epoch 9799 Generator loss: 0.8711177 Discriminator loss: 0.5563067
Epoch 9800 Generator loss: 0.8345032 Discriminator loss: 0.57877326
Epoch 9801 Generator loss: 0.756837 Discriminator loss: 0.6261751
Epoch 9802 Generator loss: 0.63713443 Discriminator loss: 0.71531326
Epoch 9803 Generator loss: 0.5001998 Discriminator loss: 0.84715784
Epoch 9804 Generator loss: 0.38747808 Discriminator loss: 0.98538303
Epoch 9805 Generator loss: 0.3161935 Discriminator loss: 1.0972642
Epoch 9806 Generator loss: 0.28151083 Discriminator loss: 1.1651312
Epoch 9807 Generator loss: 0.2887502 Discriminator loss: 1.1730874
Epoch 9808 Generator loss: 0.35388213 Discriminator loss: 1.1147192
Epoch 9809 Generator loss: 0.51450825 Discriminator loss: 0.99273014
Epoch 9810 Generator loss: 0.8146213 Discriminator loss: 0.8361556
Epoch 9811 Generator loss: 1.2867593 Discriminator loss: 0.6778753
Epoch 9812 Generator loss: 1.8545904 Discriminator loss: 0.5580431
Epoch 9813 Generator loss: 2.3739257 Discriminator lo

Epoch 9934 Generator loss: 0.61117727 Discriminator loss: 0.7398047
Epoch 9935 Generator loss: 0.5669496 Discriminator loss: 0.80222195
Epoch 9936 Generator loss: 0.62682265 Discriminator loss: 0.78007877
Epoch 9937 Generator loss: 0.73952323 Discriminator loss: 0.7380291
Epoch 9938 Generator loss: 0.83174163 Discriminator loss: 0.7277764
Epoch 9939 Generator loss: 0.89827603 Discriminator loss: 0.7259383
Epoch 9940 Generator loss: 0.91971445 Discriminator loss: 0.7254582
Epoch 9941 Generator loss: 0.8901156 Discriminator loss: 0.7316552
Epoch 9942 Generator loss: 0.90910834 Discriminator loss: 0.7180068
Epoch 9943 Generator loss: 0.93828666 Discriminator loss: 0.70487106
Epoch 9944 Generator loss: 0.9316718 Discriminator loss: 0.70356584
Epoch 9945 Generator loss: 0.91478807 Discriminator loss: 0.70591056
Epoch 9946 Generator loss: 0.8689393 Discriminator loss: 0.71699256
Epoch 9947 Generator loss: 0.7924549 Discriminator loss: 0.74162364
Epoch 9948 Generator loss: 0.7926823 Discrimin

# Proof of concept of working of non-linear-GAN with AutoEncoder and NoisyAutoEncoder

In [3]:
import torch
from model import gan

# Loss functions
criterion = torch.nn.BCELoss()

def AutoEncoderLoss(output, target):
    # MSE loss
    loss = torch.mean((output - target)**2)
    return loss


# Network param settings
embedding_dim = 8
internal_dim = 8
output_dim = 1

# Train hyperparameters
epochs = 100
languages = {'src': ['de'], 'trgt': ['en']}
nr_langs = 2
batch_size = 10
real_label = 1


# Architectures
### GAN ###
net = gan.GAN(embedding_dim, internal_dim, output_dim, languages['src'])
# Get optimizers; 1 per source language of encoder and 1 for discriminator
optimizers = {'gen': {}}
for lang in languages['src']:
    optimizers['gen'][lang] = torch.optim.Adam([{'params': net.generator.encoders[lang].parameters()},
                                                {'params': net.generator.decoder.parameters()}
                                               ],
                                               lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                               weight_decay=0.01, amsgrad=False)
optimizers['dis'] = torch.optim.Adam(net.discriminator.parameters(),
                                     lr=0.001, betas=(0.9, 0.999), eps=1e-08, 
                                     weight_decay=0.001, amsgrad=False)

### AUTO ENCODER ###
ae = AutoEncoder(embedding_dim, internal_dim, net.generator.decoder)
optimizer_ae = torch.optim.Adam(ae.parameters(),
                                lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                weight_decay=0., amsgrad=False)

### NOISY AUTO ENCODER ###
nae = NoisyAutoEncoder(net.generator.encoders['de'], embedding_dim, internal_dim, embedding_dim)
optimizer_nae = torch.optim.Adam(nae.parameters(),
                                 lr=0.01, betas=(0.9, 0.999), eps=1e-08, 
                                 weight_decay=0., amsgrad=False)



# Create toy train dataset
rows, cols = batch_size, embedding_dim
train_data = torch.rand(rows, cols)


# Report outcome before training
reproduction = net.generator(train_data, 'de')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))
print('Start training.')


# Train
for epoch in range(5000):
    
    ### GAN STEP ###
    
    # Update discriminator #
    
    net.discriminator.train()
    net.generator.eval()
    net.discriminator.zero_grad()

    # Labels
    y_true = torch.zeros([nr_langs*batch_size])
    y_true[0:batch_size] = real_label  # First elements are target embeddings


    # Predictions all-real minibatch
    y_preds = torch.zeros([nr_langs*batch_size, 1]) 
    y_preds[0:batch_size] = net.discriminator(train_data)

    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')  # Generate fake data aka translate
    y_preds[batch_size:] = net.discriminator(x_trans)
    loss_dis = criterion(y_preds, y_true)
    loss_dis.backward()    # Compute gradients only for discriminator
    optimizers['dis'].step() 
    
    # Update generator #
    
    net.generator.train()
    net.discriminator.eval()
    net.generator.zero_grad()
    
    # Labels
    y_true = torch.full((batch_size,), real_label)
    
    # Predictions all-fake minibatch
    x_trans = net.generator(train_data, 'de')
    y_pred = net.discriminator(x_trans)
    loss_gen = criterion(y_pred, y_true)
    loss_gen.backward()    # Compute gradients only for generator
    optimizers['gen']['de'].step()
    
    
    ### AUTO ENCODER STEP ###

    # Perform training
    ae.train()
    optimizer_ae.zero_grad()
    y_pred = ae(train_data)
    loss_ae = AutoEncoderLoss(y_pred, train_data)
    loss_ae.backward()
    optimizer_ae.step()


    ### NOISY AUTO ENCODER STEP ###

    # Perform training
    nae.train()
    nae.zero_grad()
    y_pred = nae(train_data)
    loss_nae = AutoEncoderLoss(y_pred, train_data)
    loss_nae.backward()
    optimizer_nae.step()
    
    
    print('Epoch', epoch, 'Generator loss:', loss_gen.detach().numpy(), 
          'Discriminator loss:', loss_dis.detach().numpy(),
          'AutoEncoder loss:', loss_ae.detach().numpy(),
          'NoisyAutoEncoder loss:', loss_nae.detach().numpy())
    
# Report outcome after training
reproduction = net.generator(train_data, 'de')
print('Training finished.')
print('Input data:\n', train_data)
print('Reproduced data:\n', reproduction)
print('MSE loss on reproducing input:', AutoEncoderLoss(reproduction, train_data))

Created: Nonlinear GAN.
Input data:
 tensor([[0.2247, 0.4349, 0.6344, 0.7456, 0.7079, 0.7276, 0.8161, 0.9895],
        [0.0425, 0.1440, 0.7051, 0.4734, 0.3901, 0.3094, 0.4554, 0.7727],
        [0.0507, 0.6168, 0.4725, 0.7723, 0.4898, 0.4439, 0.8802, 0.4298],
        [0.3774, 0.2089, 0.6967, 0.4745, 0.6275, 0.8925, 0.0507, 0.0507],
        [0.6498, 0.9873, 0.1560, 0.1569, 0.5199, 0.8265, 0.2623, 0.4122],
        [0.0332, 0.7219, 0.3819, 0.5320, 0.1252, 0.8065, 0.1154, 0.1134],
        [0.6487, 0.3832, 0.5348, 0.5797, 0.4034, 0.5564, 0.5041, 0.9185],
        [0.6818, 0.4907, 0.5059, 0.5488, 0.5640, 0.8731, 0.2173, 0.4872],
        [0.6358, 0.0464, 0.6327, 0.6932, 0.4965, 0.6510, 0.1919, 0.1215],
        [0.5229, 0.7773, 0.4770, 0.5108, 0.5309, 0.2330, 0.6492, 0.8709]])
Reproduced data:
 tensor([[-0.2268,  0.3478, -0.0257,  0.3729, -0.3257,  0.2215, -0.2555, -0.1121],
        [-0.2271,  0.3476, -0.0254,  0.3731, -0.3251,  0.2213, -0.2551, -0.1124],
        [-0.2099,  0.3623, -0.0373,  0.3

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/usr/local/lib/python3.5/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target

Epoch 23 Generator loss: 1.0705522 Discriminator loss: 0.395589 AutoEncoder loss: 0.06202997 NoisyAutoEncoder loss: 0.10011097
Epoch 24 Generator loss: 1.0502615 Discriminator loss: 0.41283268 AutoEncoder loss: 0.06128483 NoisyAutoEncoder loss: 0.099241056
Epoch 25 Generator loss: 1.0365292 Discriminator loss: 0.4292335 AutoEncoder loss: 0.06076442 NoisyAutoEncoder loss: 0.09543355
Epoch 26 Generator loss: 1.0319465 Discriminator loss: 0.44375658 AutoEncoder loss: 0.060564715 NoisyAutoEncoder loss: 0.09214412
Epoch 27 Generator loss: 1.0352122 Discriminator loss: 0.45666632 AutoEncoder loss: 0.06069445 NoisyAutoEncoder loss: 0.08373675
Epoch 28 Generator loss: 1.0507826 Discriminator loss: 0.46616015 AutoEncoder loss: 0.060971588 NoisyAutoEncoder loss: 0.07625899
Epoch 29 Generator loss: 1.0816085 Discriminator loss: 0.4710806 AutoEncoder loss: 0.061101474 NoisyAutoEncoder loss: 0.07143686
Epoch 30 Generator loss: 1.1240819 Discriminator loss: 0.472009 AutoEncoder loss: 0.060867537 Noi

Epoch 109 Generator loss: 1.4202565 Discriminator loss: 0.6407743 AutoEncoder loss: 0.028297642 NoisyAutoEncoder loss: 0.033233903
Epoch 110 Generator loss: 1.3807821 Discriminator loss: 0.64036125 AutoEncoder loss: 0.02821036 NoisyAutoEncoder loss: 0.032125108
Epoch 111 Generator loss: 1.3604903 Discriminator loss: 0.6352098 AutoEncoder loss: 0.028116468 NoisyAutoEncoder loss: 0.032148708
Epoch 112 Generator loss: 1.3542169 Discriminator loss: 0.6263014 AutoEncoder loss: 0.02804113 NoisyAutoEncoder loss: 0.033000752
Epoch 113 Generator loss: 1.3491486 Discriminator loss: 0.6181093 AutoEncoder loss: 0.027983297 NoisyAutoEncoder loss: 0.032394387
Epoch 114 Generator loss: 1.3551856 Discriminator loss: 0.60426176 AutoEncoder loss: 0.027906626 NoisyAutoEncoder loss: 0.032303758
Epoch 115 Generator loss: 1.3727795 Discriminator loss: 0.58252585 AutoEncoder loss: 0.027824303 NoisyAutoEncoder loss: 0.031951733
Epoch 116 Generator loss: 1.3943349 Discriminator loss: 0.5544828 AutoEncoder loss

Epoch 184 Generator loss: 0.9250692 Discriminator loss: 0.6339005 AutoEncoder loss: 0.013585798 NoisyAutoEncoder loss: 0.02595022
Epoch 185 Generator loss: 0.9215051 Discriminator loss: 0.63849336 AutoEncoder loss: 0.013144669 NoisyAutoEncoder loss: 0.025343016
Epoch 186 Generator loss: 0.93292683 Discriminator loss: 0.6358591 AutoEncoder loss: 0.012795361 NoisyAutoEncoder loss: 0.02542392
Epoch 187 Generator loss: 0.9616827 Discriminator loss: 0.624912 AutoEncoder loss: 0.012558961 NoisyAutoEncoder loss: 0.025136659
Epoch 188 Generator loss: 0.99073 Discriminator loss: 0.6114435 AutoEncoder loss: 0.012400933 NoisyAutoEncoder loss: 0.02544565
Epoch 189 Generator loss: 1.010496 Discriminator loss: 0.5979093 AutoEncoder loss: 0.0122874435 NoisyAutoEncoder loss: 0.025112465
Epoch 190 Generator loss: 1.0294983 Discriminator loss: 0.58140945 AutoEncoder loss: 0.012174896 NoisyAutoEncoder loss: 0.025019918
Epoch 191 Generator loss: 1.0446742 Discriminator loss: 0.5641717 AutoEncoder loss: 0.

...

Epoch 4720 Generator loss: 2.6790693 Discriminator loss: 0.10272418 AutoEncoder loss: 0.00310883 NoisyAutoEncoder loss: 0.0058355494
Epoch 4721 Generator loss: 2.6663542 Discriminator loss: 0.10253285 AutoEncoder loss: 0.0031075366 NoisyAutoEncoder loss: 0.005900559
Epoch 4722 Generator loss: 2.6223006 Discriminator loss: 0.10165663 AutoEncoder loss: 0.003105299 NoisyAutoEncoder loss: 0.005971466
Epoch 4723 Generator loss: 2.5855763 Discriminator loss: 0.10157595 AutoEncoder loss: 0.0031014872 NoisyAutoEncoder loss: 0.0058526034
Epoch 4724 Generator loss: 2.599875 Discriminator loss: 0.10026561 AutoEncoder loss: 0.0030966396 NoisyAutoEncoder loss: 0.005761237
Epoch 4725 Generator loss: 2.6407979 Discriminator loss: 0.09850934 AutoEncoder loss: 0.003091832 NoisyAutoEncoder loss: 0.0058383876
Epoch 4726 Generator loss: 2.65457 Discriminator loss: 0.09726102 AutoEncoder loss: 0.0030877325 NoisyAutoEncoder loss: 0.005906018
Epoch 4727 Generator loss: 2.6393142 Discriminator loss: 0.0969093

Epoch 4791 Generator loss: 2.7547374 Discriminator loss: 0.085752055 AutoEncoder loss: 0.0030150856 NoisyAutoEncoder loss: 0.005997614
Epoch 4792 Generator loss: 2.7218268 Discriminator loss: 0.08666347 AutoEncoder loss: 0.0030146153 NoisyAutoEncoder loss: 0.0058539314
Epoch 4793 Generator loss: 2.7261994 Discriminator loss: 0.08775517 AutoEncoder loss: 0.0030151801 NoisyAutoEncoder loss: 0.005764613
Epoch 4794 Generator loss: 2.7498221 Discriminator loss: 0.088380456 AutoEncoder loss: 0.0030140884 NoisyAutoEncoder loss: 0.0059310556
Epoch 4795 Generator loss: 2.75627 Discriminator loss: 0.08867447 AutoEncoder loss: 0.0030135342 NoisyAutoEncoder loss: 0.005848919
Epoch 4796 Generator loss: 2.7250154 Discriminator loss: 0.087668784 AutoEncoder loss: 0.0030127491 NoisyAutoEncoder loss: 0.0058977576
Epoch 4797 Generator loss: 2.6824706 Discriminator loss: 0.08699221 AutoEncoder loss: 0.003012762 NoisyAutoEncoder loss: 0.0058704996
Epoch 4798 Generator loss: 2.6580112 Discriminator loss: 0

Epoch 4863 Generator loss: 2.470191 Discriminator loss: 0.10769755 AutoEncoder loss: 0.0029543312 NoisyAutoEncoder loss: 0.0058460007
Epoch 4864 Generator loss: 2.467152 Discriminator loss: 0.10748043 AutoEncoder loss: 0.002946951 NoisyAutoEncoder loss: 0.0060449294
Epoch 4865 Generator loss: 2.465453 Discriminator loss: 0.10673343 AutoEncoder loss: 0.002942151 NoisyAutoEncoder loss: 0.0057791965
Epoch 4866 Generator loss: 2.4613378 Discriminator loss: 0.106852725 AutoEncoder loss: 0.0029377597 NoisyAutoEncoder loss: 0.0058578206
Epoch 4867 Generator loss: 2.4769711 Discriminator loss: 0.106311224 AutoEncoder loss: 0.0029355735 NoisyAutoEncoder loss: 0.005804357
Epoch 4868 Generator loss: 2.4873261 Discriminator loss: 0.106146514 AutoEncoder loss: 0.0029327532 NoisyAutoEncoder loss: 0.0058946405
Epoch 4869 Generator loss: 2.4848213 Discriminator loss: 0.106036864 AutoEncoder loss: 0.0029310577 NoisyAutoEncoder loss: 0.005829608
Epoch 4870 Generator loss: 2.4814572 Discriminator loss: 0

Epoch 4928 Generator loss: 2.449337 Discriminator loss: 0.10719248 AutoEncoder loss: 0.0028418163 NoisyAutoEncoder loss: 0.006037441
Epoch 4929 Generator loss: 2.4867444 Discriminator loss: 0.109301135 AutoEncoder loss: 0.0028467975 NoisyAutoEncoder loss: 0.0058750855
Epoch 4930 Generator loss: 2.616921 Discriminator loss: 0.107142724 AutoEncoder loss: 0.0028450051 NoisyAutoEncoder loss: 0.005778308
Epoch 4931 Generator loss: 2.659209 Discriminator loss: 0.10660943 AutoEncoder loss: 0.0028419541 NoisyAutoEncoder loss: 0.005979143
Epoch 4932 Generator loss: 2.5874577 Discriminator loss: 0.10769315 AutoEncoder loss: 0.0028371965 NoisyAutoEncoder loss: 0.0060380544
Epoch 4933 Generator loss: 2.5144005 Discriminator loss: 0.10691246 AutoEncoder loss: 0.0028342786 NoisyAutoEncoder loss: 0.0058076875
Epoch 4934 Generator loss: 2.4996393 Discriminator loss: 0.10595338 AutoEncoder loss: 0.0028294378 NoisyAutoEncoder loss: 0.005838347
Epoch 4935 Generator loss: 2.5278528 Discriminator loss: 0.1

Epoch 4999 Generator loss: 2.5714514 Discriminator loss: 0.107736684 AutoEncoder loss: 0.0031086388 NoisyAutoEncoder loss: 0.0058809044
Training finished.
Input data:
 tensor([[0.2247, 0.4349, 0.6344, 0.7456, 0.7079, 0.7276, 0.8161, 0.9895],
        [0.0425, 0.1440, 0.7051, 0.4734, 0.3901, 0.3094, 0.4554, 0.7727],
        [0.0507, 0.6168, 0.4725, 0.7723, 0.4898, 0.4439, 0.8802, 0.4298],
        [0.3774, 0.2089, 0.6967, 0.4745, 0.6275, 0.8925, 0.0507, 0.0507],
        [0.6498, 0.9873, 0.1560, 0.1569, 0.5199, 0.8265, 0.2623, 0.4122],
        [0.0332, 0.7219, 0.3819, 0.5320, 0.1252, 0.8065, 0.1154, 0.1134],
        [0.6487, 0.3832, 0.5348, 0.5797, 0.4034, 0.5564, 0.5041, 0.9185],
        [0.6818, 0.4907, 0.5059, 0.5488, 0.5640, 0.8731, 0.2173, 0.4872],
        [0.6358, 0.0464, 0.6327, 0.6932, 0.4965, 0.6510, 0.1919, 0.1215],
        [0.5229, 0.7773, 0.4770, 0.5108, 0.5309, 0.2330, 0.6492, 0.8709]])
Reproduced data:
 tensor([[ 0.6698,  0.0986,  0.6580,  0.7306,  0.6500,  0.7724,  0.2545,  